In [1]:
import os
import numpy as np
import pandas as pd
from glob import glob
from tensorflow import keras
import matplotlib.pyplot as plt

from data_generator import DataGenerator
from utils import visualize_depth_map
from metrics import Metric

In [2]:
model = keras.models.load_model('checkpoint/model_1653373548.903842/')

2022-06-07 01:10:10.892012: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
HEIGHT = 256
WIDTH = 256
LR = 0.0002
EPOCHS = 100
BATCH_SIZE = 32

df = pd.read_csv('data_df.csv')

In [ ]:
# test_loader = next(
#     iter(DataGenerator(data=df[265:].reset_index(drop="true"),
#                        batch_size=6, dim=(HEIGHT, WIDTH)))
# )
# visualize_depth_map(test_loader, test=True, model=model)

In [4]:
test_loader = DataGenerator(
    data=df[260:].reset_index(drop="true"), batch_size=16, dim=(HEIGHT, WIDTH)
)
df[260:].shape

(65, 3)

In [5]:
evaluation_metrics = {'mean_absrel': [], 'mean_sqrel': [], 'rms': [], 'rms_log': [], 'delta1': [], 'delta2': [], 'delta3': []}
for samples in test_loader:
    inp, target = samples
    pred = model.predict(inp)
    for i in range(len(inp)):
        metric = Metric(target[i], pred[i])
        evaluation_metrics['mean_absrel'].append(metric.mean_absrel_err())
        evaluation_metrics['mean_sqrel'].append(metric.mean_sqrel_err())
        evaluation_metrics['rms'].append(metric.rms_err())
        evaluation_metrics['rms_log'].append(metric.rms_log_err())
        evaluation_metrics['delta1'].append(metric.percentage_of_pixel_acc()['delta1'])
        evaluation_metrics['delta2'].append(metric.percentage_of_pixel_acc()['delta2'])
        evaluation_metrics['delta3'].append(metric.percentage_of_pixel_acc()['delta3'])

1/1 [==============================] - 0s 84ms/step


In [6]:
res_df = pd.DataFrame(evaluation_metrics)

In [7]:
res_df.mean()

mean_absrel    1.100400
mean_sqrel     0.828076
rms            0.702699
rms_log        1.468345
delta1         0.261761
delta2         0.496734
delta3         0.614590
dtype: float64